### Imports

In [47]:
import numpy as np
import pandas as pd
from case_studies import *
import time
import matplotlib.pyplot as plt

In [48]:
def backtracking_line_search(f, df, x, pk, alpha_init, c1, rho):
    
    alpha = alpha_init
    while f(x + alpha * pk) > f(x) + c1 * alpha * np.dot(df(x), pk):
        alpha *= rho
    return alpha

### Conjugate Gradients

In [ ]:
#Max iter afhænger nok af newton som kalder CG, måske skal det hedde noget andet
def conjugate_gradients(Q, g, eps, max_iter=1000):
    x = 0
    grad = g
    p = -grad
    xs = [x]
    for i in range(max_iter):
        # skal måske ik være minus
        alpha_k = -(np.dot(p.T, grad) / np.dot(p.T, np.dot(Q, p)))
        
        x = x + alpha_k * p
        
        grad = np.dot(Q, x) + g
        
        if np.linalg.norm(grad) < eps:
            break

        p = -grad + (np.dot(grad.T, np.dot(Q, p)) / np.dot(p.T, np.dot(Q, p))) * p
        
        xs.append(x)
    return x, xs

### Approximate Newton

In [ ]:
def approximate_newton(x, df, hf, c1, rho, max_iter=1000, tol=1e-6):
    xs = [x]
    for i in range(max_iter):

        if np.linalg.norm(df(x)) < tol:
            break 
        
        n_k = 0.5 * min(0.5, np.sqrt(np.linalg.norm(df(x))))
        eps = n_k * np.linalg.norm(df(x))
        p = conjugate_gradients(hf(x), df(x), eps)
        alpha = backtracking_line_search(x, p, 1.0, c1, rho)
        x = x + alpha * p
        xs.append(x)
    return x, xs

### Benchmarking

In [51]:
def benchmark(f, df, optimizer, x0, x_opt, Hf):
    start_time = time.time()

    if Hf is not None:
        x_final, xs = optimizer(f, df, Hf, x0)
    else:
        x_final, xs = optimizer(f, df, x0)

    end_time = time.time()

    num_iterations = len(xs)
    final_solution_point = x_final
    dist_to_optimum = np.linalg.norm(x_final-x_opt)
    final_fun_value = f(xs[-1])
    duration = end_time-start_time
    
    grad_norms = [np.linalg.norm(df(x)) for x in xs]

    return (num_iterations, duration, final_fun_value, final_solution_point, dist_to_optimum, grad_norms)


In [52]:
# x0 = np.random.randn(2) 
# benchmark_results = []

# for f, df, Hf, fname in zip(fs, dfs, Hfs, fnames):
#     x_optimal = x_opt(fname, len(x0))

#     sd_result = benchmark(f, df, steepest_descent, x0, x_optimal, Hf=None)
#     benchmark_results.append((fname, "Steepest Descent") + sd_result)

# for f, df, Hf, fname in zip(fs, dfs, Hfs, fnames):    
#     x_optimal = x_opt(fname, len(x0))
    
#     nm_result = benchmark(f, df, newtons_method, x0, x_optimal, Hf)
#     benchmark_results.append((fname, "Newton's Method") + nm_result)

# # Convert to DataFrame
# columns = ["Function", "Optimizer", "Iterations", "Time", "Final Function Value", "Final Solution Point", "Distance to Optimum", "Gradient Norms"]
# df_results = pd.DataFrame(benchmark_results, columns=columns)

# df_results

### Plotting

In [ ]:
# plt.figure(figsize=(12, 8))
# for i in range(5):
    # plt.plot(range(len(df_results.iloc[i,7][:100])), df_results.iloc[i,7][:100], label=f"{df_results.iloc[i,0]}")
# plt.xlabel('Steps')
# plt.ylabel('Gradient Norm')
# plt.title("Steepest Descent - Graident norms for each function")
# plt.yscale("log")
# plt.legend()
# plt.grid(True, which="both", linestyle="--", linewidth=0.5)
# plt.show()

<Figure size 1200x800 with 0 Axes>

<Figure size 1200x800 with 0 Axes>